In [ ]:
# Dataset loader
class OCRDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, labels, transform=None):
        self.img_paths = img_paths
        self.labels = labels
        self.transform = transform

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert('L')
        if self.transform:
            img = self.transform(img)
        label = self.labels[idx]
        return img, label

    def __len__(self):
        return len(self.img_paths)

# Training loop (simplified)
for epoch in range(num_epochs):
    for imgs, labels in dataloader:
        preds = model(imgs)
        loss = ctc_loss(preds, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(num_epochs):
    model.train()
    for imgs, labels, label_lengths in train_loader:
        preds = model(imgs)  # [T, N, C] for CTC
        pred_lengths = torch.full(size=(preds.size(1),), 
                                  fill_value=preds.size(0), 
                                  dtype=torch.long)
        
        loss = ctc_loss(preds.log_softmax(2), labels, pred_lengths, label_lengths)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
        optimizer.step()
    
    # Validation
    model.eval()
    with torch.no_grad():
        val_loss, cer, wer = evaluate(model, val_loader)